# 실전 프로젝트: 타이타닉 생존 예측 (Kaggle 스타일)

실제 데이터셋을 사용하여 머신러닝 프로젝트를 처음부터 끝까지 수행합니다. Kaggle 경진대회 방식으로 접근하여 실무 노하우를 익힙니다.

**학습 목표:**
- 완전한 ML 워크플로우 경험
- 탐색적 데이터 분석 (EDA) 수행
- 특성 엔지니어링 기법 적용
- 여러 모델 비교 및 선택
- 하이퍼파라미터 튜닝
- Kaggle 경진대회 전략 이해

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_auc_score, roc_curve
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. 문제 정의

**목표**: 타이타닉 승객의 생존 여부를 예측하는 분류 모델 개발

**평가 지표**: Accuracy (정확도)

**데이터**: 승객의 나이, 성별, 객실 등급, 요금 등의 정보

## 2. 데이터 로드 및 기본 탐색

In [ ]:
# seaborn 내장 타이타닉 데이터셋 사용
# Kaggle에서는 train.csv, test.csv를 다운로드하여 사용
df = sns.load_dataset('titanic')

print("=== 데이터 기본 정보 ===")
print(f"데이터 형상: {df.shape}")
print(f"\n컬럼 목록:")
print(df.columns.tolist())
print(f"\n데이터 타입:")
print(df.dtypes)

In [ ]:
# 처음 몇 행 확인
print("처음 5행:")
df.head()

In [ ]:
# 기술 통계
print("기술 통계:")
df.describe()

In [ ]:
# 타겟 변수 분포
print("=== 생존 여부 분포 ===")
print(df['survived'].value_counts())
print(f"\n생존 비율:")
print(df['survived'].value_counts(normalize=True))

# 시각화
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

df['survived'].value_counts().plot(kind='bar', ax=ax[0])
ax[0].set_title('Survival Count')
ax[0].set_xlabel('Survived (0=No, 1=Yes)')
ax[0].set_ylabel('Count')

df['survived'].value_counts(normalize=True).plot(kind='pie', autopct='%1.1f%%', ax=ax[1])
ax[1].set_title('Survival Proportion')
ax[1].set_ylabel('')

plt.tight_layout()
plt.show()

## 3. 탐색적 데이터 분석 (EDA)

### 3.1 결측치 분석

In [ ]:
# 결측치 확인
print("=== 결측치 분석 ===")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    '결측치 수': missing,
    '결측치 비율(%)': missing_pct
})
print(missing_df[missing_df['결측치 수'] > 0].sort_values(by='결측치 수', ascending=False))

# 시각화
plt.figure(figsize=(10, 6))
missing_data = missing_df[missing_df['결측치 수'] > 0].sort_values(by='결측치 수', ascending=False)
plt.barh(missing_data.index, missing_data['결측치 비율(%)'])
plt.xlabel('Missing Percentage (%)')
plt.title('Missing Values by Feature')
plt.tight_layout()
plt.show()

### 3.2 범주형 변수와 생존의 관계

In [ ]:
# 주요 범주형 변수와 생존의 관계
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 성별
sns.countplot(data=df, x='sex', hue='survived', ax=axes[0, 0])
axes[0, 0].set_title('Survival by Sex')

# 객실 등급
sns.countplot(data=df, x='pclass', hue='survived', ax=axes[0, 1])
axes[0, 1].set_title('Survival by Class')

# 승선 항구
sns.countplot(data=df, x='embarked', hue='survived', ax=axes[0, 2])
axes[0, 2].set_title('Survival by Embarked')

# 형제/배우자 수
sns.countplot(data=df, x='sibsp', hue='survived', ax=axes[1, 0])
axes[1, 0].set_title('Survival by SibSp')

# 부모/자녀 수
sns.countplot(data=df, x='parch', hue='survived', ax=axes[1, 1])
axes[1, 1].set_title('Survival by Parch')

# 혼자 여행 여부
df['alone'] = ((df['sibsp'] + df['parch']) == 0).astype(int)
sns.countplot(data=df, x='alone', hue='survived', ax=axes[1, 2])
axes[1, 2].set_title('Survival by Alone')

plt.tight_layout()
plt.show()

In [ ]:
# 생존율 통계
print("=== 범주별 생존율 ===")
print("\n성별:")
print(df.groupby('sex')['survived'].mean())
print("\n객실 등급:")
print(df.groupby('pclass')['survived'].mean())
print("\n승선 항구:")
print(df.groupby('embarked')['survived'].mean())

### 3.3 수치형 변수 분석

In [ ]:
# 나이와 요금 분포 (생존 여부별)
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 나이 분포
for survived in [0, 1]:
    axes[0, 0].hist(df[df['survived'] == survived]['age'].dropna(), 
                    bins=30, alpha=0.5, label=f'Survived={survived}')
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Age Distribution by Survival')
axes[0, 0].legend()

# 나이 박스플롯
sns.boxplot(data=df, x='survived', y='age', ax=axes[0, 1])
axes[0, 1].set_title('Age by Survival')

# 요금 분포 (로그 스케일)
for survived in [0, 1]:
    axes[1, 0].hist(np.log1p(df[df['survived'] == survived]['fare'].dropna()), 
                    bins=30, alpha=0.5, label=f'Survived={survived}')
axes[1, 0].set_xlabel('Log(Fare + 1)')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Fare Distribution by Survival (Log Scale)')
axes[1, 0].legend()

# 요금 박스플롯
sns.boxplot(data=df, x='survived', y='fare', ax=axes[1, 1])
axes[1, 1].set_title('Fare by Survival')
axes[1, 1].set_ylim(0, 300)

plt.tight_layout()
plt.show()

In [ ]:
# 상관관계 분석
print("=== 수치형 변수 상관관계 ===")
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

print("\n타겟(survived)과의 상관관계:")
print(correlation['survived'].sort_values(ascending=False))

## 4. 데이터 전처리 및 특성 엔지니어링

In [ ]:
# 작업용 데이터 복사
df_clean = df.copy()

print("=== 전처리 시작 ===")
print(f"초기 데이터 형상: {df_clean.shape}")

### 4.1 불필요한 컬럼 제거

In [ ]:
# 중복되거나 불필요한 컬럼 제거
drop_cols = ['deck', 'embark_town', 'alive', 'who', 'adult_male', 'class']
df_clean = df_clean.drop(columns=drop_cols, errors='ignore')

print(f"컬럼 제거 후: {df_clean.shape}")
print(f"남은 컬럼: {df_clean.columns.tolist()}")

### 4.2 결측치 처리

In [ ]:
# 나이: 중간값으로 대체
age_median = df_clean['age'].median()
df_clean['age'] = df_clean['age'].fillna(age_median)
print(f"나이 결측치를 중간값({age_median})으로 대체")

# 승선 항구: 최빈값으로 대체
embarked_mode = df_clean['embarked'].mode()[0]
df_clean['embarked'] = df_clean['embarked'].fillna(embarked_mode)
print(f"승선 항구 결측치를 최빈값({embarked_mode})으로 대체")

# 요금: 중간값으로 대체
fare_median = df_clean['fare'].median()
df_clean['fare'] = df_clean['fare'].fillna(fare_median)

print(f"\n결측치 처리 후:")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])

### 4.3 특성 엔지니어링

도메인 지식을 활용하여 새로운 특성을 생성합니다.

In [ ]:
# 1. 가족 크기
df_clean['family_size'] = df_clean['sibsp'] + df_clean['parch'] + 1
print("가족 크기 특성 생성: sibsp + parch + 1")

# 2. 혼자 여행 여부
df_clean['is_alone'] = (df_clean['family_size'] == 1).astype(int)
print("혼자 여행 여부 특성 생성")

# 3. 나이 그룹
df_clean['age_group'] = pd.cut(df_clean['age'],
                                bins=[0, 12, 18, 35, 60, 100],
                                labels=['Child', 'Teen', 'Young', 'Middle', 'Senior'])
print("나이 그룹 특성 생성")

# 4. 요금 구간
df_clean['fare_bin'] = pd.qcut(df_clean['fare'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])
print("요금 구간 특성 생성")

# 5. 호칭 추출 (선택적)
# df_clean['title'] = df_clean['name'].str.extract(' ([A-Za-z]+)\.', expand=False)

print(f"\n특성 엔지니어링 후 형상: {df_clean.shape}")

In [ ]:
# 새로운 특성과 생존의 관계 확인
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sns.countplot(data=df_clean, x='family_size', hue='survived', ax=axes[0])
axes[0].set_title('Survival by Family Size')

sns.countplot(data=df_clean, x='age_group', hue='survived', ax=axes[1])
axes[1].set_title('Survival by Age Group')
axes[1].tick_params(axis='x', rotation=45)

sns.countplot(data=df_clean, x='fare_bin', hue='survived', ax=axes[2])
axes[2].set_title('Survival by Fare Bin')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 4.4 범주형 변수 인코딩

In [ ]:
# LabelEncoder 사용
le = LabelEncoder()

df_clean['sex'] = le.fit_transform(df_clean['sex'])
df_clean['embarked'] = le.fit_transform(df_clean['embarked'])
df_clean['age_group'] = le.fit_transform(df_clean['age_group'])
df_clean['fare_bin'] = le.fit_transform(df_clean['fare_bin'])

print("범주형 변수 인코딩 완료")
print(f"\n인코딩 후 데이터 타입:")
print(df_clean.dtypes)

### 4.5 최종 특성 선택

In [ ]:
# 모델링에 사용할 특성 선택
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
            'embarked', 'family_size', 'is_alone', 'age_group', 'fare_bin']

X = df_clean[features]
y = df_clean['survived']

print(f"최종 특성: {features}")
print(f"X 형상: {X.shape}")
print(f"y 분포: {y.value_counts().to_dict()}")

## 5. 모델링

### 5.1 데이터 분할

In [ ]:
# Train/Test 분할 (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"학습 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")
print(f"\n학습 데이터 타겟 분포: {y_train.value_counts().to_dict()}")
print(f"테스트 데이터 타겟 분포: {y_test.value_counts().to_dict()}")

In [ ]:
# 스케일링 (선형 모델용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료")

### 5.2 Baseline 모델

간단한 모델로 기준선을 설정합니다.

In [ ]:
# 기준선: 항상 다수 클래스 예측
baseline_pred = np.zeros(len(y_test))  # 모두 0 (사망) 예측
baseline_acc = accuracy_score(y_test, baseline_pred)

print(f"Baseline 정확도 (항상 사망 예측): {baseline_acc:.4f}")
print("\n이 값보다 높은 성능을 목표로 합니다.")

### 5.3 여러 모델 비교

In [ ]:
# 다양한 모델 정의
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(random_state=42)
}

# 모델 비교
print("=== 모델 비교 (5-Fold Cross Validation) ===")
results = []

for name, model in models.items():
    # 선형 모델은 스케일링된 데이터 사용
    if name in ['Logistic Regression', 'SVM']:
        X_tr, X_te = X_train_scaled, X_test_scaled
    else:
        X_tr, X_te = X_train, X_test
    
    # 교차 검증
    cv_scores = cross_val_score(model, X_tr, y_train, cv=5, scoring='accuracy')
    
    # 학습 및 테스트
    model.fit(X_tr, y_train)
    test_score = model.score(X_te, y_test)
    
    results.append({
        'Model': name,
        'CV Mean': cv_scores.mean(),
        'CV Std': cv_scores.std(),
        'Test Score': test_score
    })
    
    print(f"{name}:")
    print(f"  CV = {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")
    print(f"  Test = {test_score:.4f}")
    print()

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='CV Mean', ascending=False)
print("\n모델 순위:")
print(results_df)

In [ ]:
# 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# CV 점수
axes[0].barh(results_df['Model'], results_df['CV Mean'])
axes[0].set_xlabel('CV Accuracy')
axes[0].set_title('Cross-Validation Scores')
axes[0].set_xlim(0.7, 0.9)

# Test 점수
axes[1].barh(results_df['Model'], results_df['Test Score'])
axes[1].set_xlabel('Test Accuracy')
axes[1].set_title('Test Scores')
axes[1].set_xlim(0.7, 0.9)

plt.tight_layout()
plt.show()

### 5.4 하이퍼파라미터 튜닝

최고 성능 모델에 대해 하이퍼파라미터를 튜닝합니다.

In [ ]:
# Random Forest 튜닝
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(
    rf, rf_param_grid, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1, 
    verbose=1
)

print("Grid Search 시작...")
grid_search.fit(X_train, y_train)

print("\n=== 하이퍼파라미터 튜닝 결과 ===")
print(f"최적 파라미터: {grid_search.best_params_}")
print(f"최적 CV 점수: {grid_search.best_score_:.4f}")
print(f"테스트 점수: {grid_search.score(X_test, y_test):.4f}")

best_model = grid_search.best_estimator_

## 6. 모델 평가

### 6.1 분류 성능 지표

In [ ]:
# 예측
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# 분류 리포트
print("=== 분류 리포트 ===")
print(classification_report(y_test, y_pred, target_names=['Not Survived', 'Survived']))

# ROC AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC AUC Score: {roc_auc:.4f}")

### 6.2 혼동 행렬

In [ ]:
# 혼동 행렬 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 혼동 행렬
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Not Survived', 'Survived'],
            yticklabels=['Not Survived', 'Survived'],
            ax=axes[0])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix')

# ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
axes[1].plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})')
axes[1].plot([0, 1], [0, 1], 'k--', label='Random')
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

### 6.3 특성 중요도

In [ ]:
# 특성 중요도
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.bar(range(len(importances)), importances[indices])
plt.xticks(range(len(importances)), [features[i] for i in indices], rotation=45)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

print("\n특성 중요도 순위:")
for i in indices:
    print(f"  {features[i]:15s}: {importances[i]:.4f}")

### 6.4 오류 분석

In [ ]:
# 잘못 예측된 케이스 분석
X_test_df = X_test.copy()
X_test_df['actual'] = y_test.values
X_test_df['predicted'] = y_pred
X_test_df['correct'] = X_test_df['actual'] == X_test_df['predicted']

print("=== 예측 결과 ===")
print(f"정확히 예측: {X_test_df['correct'].sum()} / {len(X_test_df)}")
print(f"잘못 예측: {(~X_test_df['correct']).sum()} / {len(X_test_df)}")

# False Positive와 False Negative
fp = X_test_df[(X_test_df['actual'] == 0) & (X_test_df['predicted'] == 1)]
fn = X_test_df[(X_test_df['actual'] == 1) & (X_test_df['predicted'] == 0)]

print(f"\nFalse Positive (실제 사망, 예측 생존): {len(fp)}")
print(f"False Negative (실제 생존, 예측 사망): {len(fn)}")

print("\nFalse Negative 샘플 (처음 5개):")
print(fn.head())

## 7. Kaggle 경진대회 전략

### 7.1 앙상블 기법

In [ ]:
# 여러 모델의 예측을 결합
def simple_blend(models, X_train, y_train, X_test, weights=None):
    """간단한 블렌딩 앙상블"""
    if weights is None:
        weights = [1/len(models)] * len(models)
    
    predictions = np.zeros(len(X_test))
    
    for model, weight in zip(models, weights):
        model.fit(X_train, y_train)
        pred_proba = model.predict_proba(X_test)[:, 1]
        predictions += weight * pred_proba
    
    return (predictions > 0.5).astype(int)


# 앙상블 모델
ensemble_models = [
    RandomForestClassifier(n_estimators=200, random_state=42),
    GradientBoostingClassifier(n_estimators=100, random_state=42),
    LogisticRegression(max_iter=1000, random_state=42)
]

# 세 번째 모델은 스케일링된 데이터 사용
y_pred_ensemble = simple_blend(
    [ensemble_models[0], ensemble_models[1]], 
    X_train, y_train, X_test
)

# 평가
ensemble_acc = accuracy_score(y_test, y_pred_ensemble)
print(f"앙상블 정확도: {ensemble_acc:.4f}")
print(f"최고 단일 모델 정확도: {best_model.score(X_test, y_test):.4f}")
print(f"향상: {(ensemble_acc - best_model.score(X_test, y_test)):.4f}")

### 7.2 Kaggle 제출 파일 형식

In [ ]:
# Kaggle 제출용 예측 생성 (실제 Kaggle에서는 test.csv 사용)
# 여기서는 예시로 테스트 데이터 사용

submission = pd.DataFrame({
    'PassengerId': range(1, len(y_pred) + 1),  # 실제로는 test.csv의 PassengerId 사용
    'Survived': y_pred
})

print("제출 파일 형식:")
print(submission.head(10))

# CSV로 저장
# submission.to_csv('titanic_submission.csv', index=False)
# print("\nsubmission.csv 저장 완료")

## 8. Kaggle 필수 팁

### 8.1 경진대회 체크리스트

**1. 빠른 시작**
- Baseline 코드 실행하여 첫 제출
- 리더보드 위치 확인

**2. EDA 집중**
- 데이터 이해가 핵심
- 결측치, 이상치, 분포 파악
- 타겟과의 관계 분석

**3. 특성 엔지니어링**
- 도메인 지식 활용
- 교차 특성 생성 (예: family_size)
- 그룹별 통계량 (예: 그룹별 평균)

**4. 다양한 모델 시도**
- 선형 모델 → 트리 기반 → 앙상블
- 하이퍼파라미터 튜닝

**5. 앙상블**
- 다른 모델 예측 결합
- 블렌딩, 스태킹

**6. 검증 전략**
- 로컬 CV와 리더보드 점수 일치 확인
- 과적합 주의 (Public LB에 맞추지 말 것)

### 8.2 교차 검증 전략

In [ ]:
def cross_validate_model(model, X, y, n_splits=5, stratified=True):
    """
    교차 검증 수행
    
    Parameters:
    -----------
    model : sklearn estimator
    X : features
    y : target
    n_splits : 폴드 수
    stratified : 계층화 여부
    """
    if stratified:
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    else:
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train_fold = X.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_train_fold = y.iloc[train_idx]
        y_val_fold = y.iloc[val_idx]
        
        model.fit(X_train_fold, y_train_fold)
        score = model.score(X_val_fold, y_val_fold)
        scores.append(score)
        
        print(f"Fold {fold+1}: {score:.4f}")
    
    print(f"\nMean: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")
    return np.mean(scores)


# 사용 예시
print("=== Random Forest 교차 검증 ===")
cv_score = cross_validate_model(
    RandomForestClassifier(n_estimators=100, random_state=42),
    X, y, n_splits=5
)

## 요약

### 프로젝트 워크플로우

1. **문제 정의**: 목표와 평가 지표 설정
2. **데이터 탐색**: EDA로 데이터 이해
3. **전처리**: 결측치 처리, 인코딩
4. **특성 엔지니어링**: 도메인 지식 활용
5. **모델링**: 여러 모델 비교
6. **튜닝**: 하이퍼파라미터 최적화
7. **평가**: 다양한 지표로 성능 평가
8. **앙상블**: 여러 모델 결합

### 핵심 포인트

- **EDA가 가장 중요**: 데이터 이해 없이는 좋은 모델을 만들 수 없음
- **특성 엔지니어링**: 모델 성능 향상의 핵심
- **교차 검증**: 과적합 방지와 일반화 성능 확인
- **앙상블**: 다양한 모델 결합으로 성능 향상
- **반복 개선**: 한 번에 완벽한 모델은 없음, 지속적 개선이 필요